# 🧠 HyQe RAG: Hypothetical Query Embedding Retrieval-Augmented Generation

This project implements the **HyQe RAG** (Hypothetical Query Embedding Retrieval-Augmented Generation) technique, inspired by and following the methodology described in the research paper:  
[HyQe: Hypothetical Query Embeddings for Enhanced Retrieval-Augmented Generation](https://arxiv.org/pdf/2410.15262) 📄.

---

## 🚀 Overview

**HyQe RAG** enhances traditional RAG pipelines by generating hypothetical queries for each document chunk, embedding them, and using these embeddings to improve retrieval relevance. This approach enables more robust and context-aware retrieval, especially for complex or ambiguous user queries.

---

## 🛠️ Implementation Details

- **Document Loading & Splitting:**  
    Documents are loaded from the `data` directory and split into manageable chunks using a sentence splitter.

- **Embedding Models:**  
    - Uses Google GenAI for both LLM (for hypothetical query generation) and embedding models.
    - Embeddings are generated for both user queries and hypothetical queries.

- **Hypothetical Query Generation:**  
    For each document chunk, the LLM generates diverse, short hypothetical questions that could be answered by the chunk. These are then embedded and paired with their source chunk.

- **Retrieval & Re-ranking:**  
    - At query time, the user query is embedded.
    - Each document chunk is scored based on its original retrieval score and its maximum similarity to any of its hypothetical queries.
    - Chunks are re-ranked using a weighted combination of these scores.

- **Answer Generation:**  
    The top-ranked chunks are used as context for the LLM to generate a final answer.

---

## 📦 Key Features

- **Enhanced Retrieval:**  
    By leveraging hypothetical queries, the system retrieves more relevant and contextually appropriate chunks.

- **Plug-and-Play:**  
    Easily adaptable to different embedding models and LLMs.

- **Caching:**  
    Embedding pairs are cached for efficiency.

---

## 📋 Usage

1. **Configure API Keys:**  
     Set your Google API key in the environment.

2. **Load and Index Documents:**  
     Documents are loaded, split, and indexed with hypothetical query embeddings.

3. **Query:**  
     Enter a natural language question. The system retrieves, re-ranks, and generates an answer using the HyQe RAG pipeline.

---

## 📚 Reference

This implementation is directly inspired by and follows the methodology from:  
**HyQe: Hypothetical Query Embeddings for Enhanced Retrieval-Augmented Generation**  
[https://arxiv.org/pdf/2410.15262](https://arxiv.org/pdf/2410.15262)

---

## ✨ Acknowledgements

- Thanks to the authors of the HyQe paper for their innovative approach.
- Built using [LangChain](https://python.langchain.com/), [LlamaIndex](https://www.llamaindex.ai/), and Google GenAI APIs.

---

> 💡 **Note:** This notebook is a research-inspired implementation and may require further tuning for production use.  
> For more details, see the referenced paper above.

In [ ]:
import os
GOOGLE_API_KEY = ""
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

In [ ]:
from llama_index.core import VectorStoreIndex, Settings, SimpleDirectoryReader
from llama_index.core.node_parser import SentenceSplitter
from llama_index.embeddings.google_genai import GoogleGenAIEmbedding
documents = SimpleDirectoryReader("data").load_data()
splitter = SentenceSplitter(chunk_size=1000,chunk_overlap=200)
nodes = splitter.get_nodes_from_documents(documents)
embed_model = GoogleGenAIEmbedding(
    model_name="text-embedding-004",
    embed_batch_size=100,
    api_key=""
)
Settings.embed_model = embed_model
Settings.llm = llm
vector_index = VectorStoreIndex(nodes)
retriever = vector_index.as_retriever(similarity_top_k=5)

In [3]:
nodes=retriever.retrieve("what is a COCOMO model ?")

In [12]:
for node in nodes:
  print(node.text)
  print(node.score)
  print("")


35
COCOMO Model (CONT.)
Software cost estimation is done 
through three stages: 
Basic COCOMO,
Intermediate COCOMO,  
Complete COCOMO.
0.7118802550909354

31
COCOMO Model
COCOMO (COnstructive COst MOdel) 
proposed by Boehm.
Divides software product 
developments into 3 categories: 
Organic 
Semidetached 
Embedded
0.7005425318177478

40
Basic COCOMO Model (CONT .)
Effort is 
somewhat 
super-linear in 
problem  size. 
Effort
Size
0.6912063002884875

42
Basic COCOMO Model (CONT.)
Development time does not 
increase linearly with product size:
For larger products more parallel 
activities can be identified:
can be carried out simultaneously by a 
number of engineers.
0.6894930106522041

50
Shortcoming of  basic and 
intermediate COCOMO models
Both models:
consider a software product as a single 
homogeneous entity:
However, most large systems are made up of 
several smaller sub-systems.
Some sub-systems may be considered as organic 
type, some may be considered embedded, etc.
for some the 

In [ ]:
embedding_pairs = []
def generate_hypothetical_query_embeddings(nodes, llm, embed_model):
    global embedding_pairs

    for node in nodes:
        prompt = f"""
        Which kinds of questions can be answered based on the following passage?

        Passage:
        \"\"\"{node.text}\"\"\"

        Questions must be short, diverse, and each written on a separate line.
        If the passage provides no meaningful content, respond with 'No Content'.
        """

        response = llm.invoke(prompt)
        response_text = response if isinstance(response, str) else response.content

        if "No Content" in response_text:
            continue

        questions = [q.strip() for q in response_text.split('\n') if q.strip()]
        if not questions:
            continue

        question_embeddings = embed_model.get_text_embedding_batch(questions)
        for embedding in question_embeddings:
            embedding_pairs.append((embedding, node))  # or store question too if needed

    return embedding_pairs

In [5]:
embedding_pairs= generate_hypothetical_query_embeddings(nodes,llm,embed_model)

In [ ]:
from collections import defaultdict
import numpy as np
def list_to_embedding_map(embedding_pairs):
    emb_map = defaultdict(list)
    for emb, node in embedding_pairs:
        emb_map[node.node_id].append(np.array(emb))
    return emb_map

In [7]:
print(len(embedding_pairs))

25


In [ ]:
import pickle

# Save
with open("hypo_cache.pkl", "wb") as f:
    pickle.dump(embedding_pairs, f)

# Load
with open("hypo_cache.pkl", "rb") as f:
    embedding_pairs = pickle.load(f)


In [ ]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

def compute_final_scores(nodes, embedding_pairs, query_embedding, lamda=0.75):
    # Normalize query embedding for the cosine similarity
    query_vec = np.array(query_embedding).reshape(1, -1)
    query_vec /= np.linalg.norm(query_vec)
    from collections import defaultdict
    hypo_map = defaultdict(list)
    for emb, src_node in embedding_pairs:
        if src_node.node_id:  # in case node has ID
            hypo_map[src_node.node_id].append(np.array(emb))
    # For each node, find its maximum similarityy to any hypothetical query
    for node in nodes:
        node_id = node.node_id
        max_sim = 0
        if node_id in hypo_map:
            hq_embs = np.array(hypo_map[node_id])
            hq_embs = hq_embs / np.linalg.norm(hq_embs, axis=1, keepdims=True)  # normalize
            sims = cosine_similarity(query_vec, hq_embs)[0]
            max_sim = np.max(sims)
        # Final score
        node.score = node.score + lamda * max_sim

    # Sort and return
    nodes.sort(key=lambda x: x.score, reverse=True)
    return nodes

In [9]:
query="what is a COCOMO model ?"
query_embedding=embed_model.get_text_embedding(query)

In [10]:
nodes_final=compute_final_scores(nodes,embedding_pairs,query_embedding)

In [ ]:
for _node in nodes_final:
  print(_node.text)
  print(_node.score)
  print("")
# 31 ,35 , 50 , 40, 42  initial rankings
# 35 , 50 , 35 , 40 , 42  final rankings after re-ranking

31
COCOMO Model
COCOMO (COnstructive COst MOdel) 
proposed by Boehm.
Divides software product 
developments into 3 categories: 
Organic 
Semidetached 
Embedded
1.4192215507775896

50
Shortcoming of  basic and 
intermediate COCOMO models
Both models:
consider a software product as a single 
homogeneous entity:
However, most large systems are made up of 
several smaller sub-systems.
Some sub-systems may be considered as organic 
type, some may be considered embedded, etc.
for some the reliability requirements may be high, 
and so on.
1.3458826537200281

35
COCOMO Model (CONT.)
Software cost estimation is done 
through three stages: 
Basic COCOMO,
Intermediate COCOMO,  
Complete COCOMO.
1.3343138428704036

40
Basic COCOMO Model (CONT .)
Effort is 
somewhat 
super-linear in 
problem  size. 
Effort
Size
1.314182786440845

42
Basic COCOMO Model (CONT.)
Development time does not 
increase linearly with product size:
For larger products more parallel 
activities can be identified:
can be carri

In [ ]:
def generate_answer(nodes_final):
 top_k = 5
 top_chunks = nodes_final[:top_k]
 context = "\n\n".join([node.text for node in top_chunks])
 prompt = f"""Answer the following question using the context provided.

 Question:
 {query}

 Context:
 {context}

 Answer:"""

 answer = llm.invoke(prompt)
 return answer

In [13]:
answer=generate_answer(nodes_final)
print(answer.content)

The COCOMO model, which stands for COnstructive COst MOdel, was proposed by Boehm. It is used for software cost estimation and divides software product developments into three categories: Organic, Semidetached, and Embedded. Software cost estimation through COCOMO is done in three stages: Basic COCOMO, Intermediate COCOMO, and Complete COCOMO. The model suggests that effort is somewhat super-linear in problem size, and development time does not increase linearly with product size.


In [44]:
query="what is a rayleigh curve and what is its significance ?"
nodes=retriever.retrieve(query)
embedding_pairs= generate_hypothetical_query_embeddings(nodes,llm,embed_model)
query_embedding=embed_model.get_text_embedding(query)
nodes_final=compute_final_scores(nodes,embedding_pairs,query_embedding)
answer=generate_answer(nodes_final)
print(answer.content)

A Rayleigh curve represents the number of full-time personnel required at any time during a development project. It is specified by two parameters: `td`, the time at which the curve reaches its maximum, and `K`, the total area under the curve. The curve shows that a very small number of engineers are needed at the beginning of a project for planning and specification, and as the project progresses, the number of engineers slowly increases and reaches a peak. After this peak, the number of project staff falls.

Its significance is that it models the non-constant staffing levels required for development projects. Norden observed in 1958 that it represents the number of full-time personnel needed. The time `td` when the Rayleigh curve reaches its maximum value corresponds to system testing and product release. It also indicates that approximately 40% of the area under the curve (representing effort/personnel) is to the left of `td` and 60% is to the right.


In [51]:
query="describe Putnam’s Work"
nodes=retriever.retrieve(query)
embedding_pairs= generate_hypothetical_query_embeddings(nodes,llm,embed_model)
query_embedding=embed_model.get_text_embedding(query)
nodes_final=compute_final_scores(nodes,embedding_pairs,query_embedding)
answer=generate_answer(nodes_final)
print(answer)

Putnam's work, beginning in 1976, focused on the problem of staffing software projects. He observed that the level of effort in software development efforts has a similar envelope and found that the Rayleigh-Norden curve relates the number of delivered lines of code to effort and development time.

Through analysis of numerous army projects, Putnam derived the expression: L=(Ck K^(1/3) td^(4/3)). In this formula, L represents the size in KLOC, K is the effort expended, and td is the time to develop the software. Ck is a state of technology constant that reflects factors affecting programmer productivity, with values ranging from 2 for a poor development environment (no methodology, poor documentation) to 8 for a good one (software engineering principles used), and 11 for an excellent environment.

Putnam also observed that the Rayleigh curve reaches its maximum value at the time of system testing and product release, after which project staff numbers decrease until product installation

In [58]:
query="explain Delphi Estimation"
nodes=retriever.retrieve(query)
embedding_pairs= generate_hypothetical_query_embeddings(nodes,llm,embed_model)
query_embedding=embed_model.get_text_embedding(query)
nodes_final=compute_final_scores(nodes,embedding_pairs,query_embedding)
answer=generate_answer(nodes_final)
print(answer.content)

Delphi Estimation is a heuristic technique used for cost estimation. It involves a team of experts and a coordinator. Experts carry out estimations independently, mentioning the rationale behind their estimation. The coordinator notes down any extraordinary rationale and circulates it among the experts. Experts then re-estimate. A key characteristic is that experts never meet each other to discuss their viewpoints. This method overcomes some of the problems of individual bias found in simple expert judgment.


In [57]:
query="tell about Heuristic techniques discussed in the PDF"
nodes=retriever.retrieve(query)
embedding_pairs= generate_hypothetical_query_embeddings(nodes,llm,embed_model)
query_embedding=embed_model.get_text_embedding(query)
nodes_final=compute_final_scores(nodes,embedding_pairs,query_embedding)
answer=generate_answer(nodes_final)
print(answer.content)

Heuristic techniques are defined as an educated guess based on past experience, or systematic guesses by experts.

The specific heuristic techniques discussed are:

*   **Expert Judgement:** This is described as an euphemism for a guess made by an expert and suffers from individual bias.
*   **Delphi Estimation:** This technique is mentioned as overcoming some of the problems associated with expert judgement.


In [59]:
query="what are different Empirical Estimation Techniques ?"
nodes=retriever.retrieve(query)
embedding_pairs= generate_hypothetical_query_embeddings(nodes,llm,embed_model)
query_embedding=embed_model.get_text_embedding(query)
nodes_final=compute_final_scores(nodes,embedding_pairs,query_embedding)
answer=generate_answer(nodes_final)
print(answer.content)

Based on the context provided, the different Empirical Estimation Techniques are:

*   **Single Variable Model**
*   **Multivariable Model**
*   **COCOMO** (mentioned as an example of an empirical technique)


In [60]:
query="what is Function Point Metric? and how is it useful in software engineering ?"
nodes=retriever.retrieve(query)
embedding_pairs= generate_hypothetical_query_embeddings(nodes,llm,embed_model)
query_embedding=embed_model.get_text_embedding(query)
nodes_final=compute_final_scores(nodes,embedding_pairs,query_embedding)
answer=generate_answer(nodes_final)
print(answer.content)

Function Point Metric (FP) is a software size estimation metric proposed by Albrecht at IBM in 1979. It was developed to overcome some of the shortcomings of the Lines of Code (LOC) metric.

**How it is useful in software engineering:**
*   It overcomes some of the shortcomings of the LOC metric.
*   Proponents claim it is language independent, meaning it can be applied regardless of the programming language used.
*   Proponents also claim that the software size can be easily derived from the problem description, making it useful in early stages of development.

The metric is calculated by counting various functional components of a software system, such as inputs, outputs, inquiries, files, and interfaces. These counts can then be multiplied by predefined weights (which vary based on the complexity of each component – Low, Average, High) and summed to arrive at the Function Point value. For example, a simplified formula is FP = 4 #inputs + 5 #Outputs + 4 #inquiries + 10 #files + 7 #in

In [61]:
query="tell about Jensen Model? compare it to other models in the PDF"
nodes=retriever.retrieve(query)
embedding_pairs= generate_hypothetical_query_embeddings(nodes,llm,embed_model)
query_embedding=embed_model.get_text_embedding(query)
nodes_final=compute_final_scores(nodes,embedding_pairs,query_embedding)
answer=generate_answer(nodes_final)
print(answer.content)

Based on the context provided:

**Jensen Model:**
The Jensen model is an estimation technique that is very similar to the Putnam model. It aims to soften the effect of schedule compression on effort, making it applicable to smaller and medium-sized projects. Jensen proposed the equation:
`L = Cte * td * K^(1/2)`
Where:
*   `L` is the lines of code (implicitly, as it's the output of the equation).
*   `Cte` is the effective technology constant.
*   `td` is the time to develop the software.
*   `K` is the effort needed to develop the software.

**Comparison to other models in the PDF:**

*   **Compared to Putnam Model:** The Jensen model is explicitly stated to be "very similar to Putnam model."
*   **Compared to Empirical Estimation Techniques (Single Variable and Multivariable Models):**
    *   The Jensen model is an empirical estimation technique that uses an equation with multiple variables (`Cte`, `td`, `K`) to estimate `L`.
    *   **Single Variable Model:** Estimates a parameter 